# Preload precipitation files

Since loading from the original COSMO GRIB files takes quite a long time, I will try to preload them in a separate file. This should hopefully speed up my analysis.

In [1]:
# Imports
from helpers import *
from config import *
import os
import numpy as np
import datetime as dt
from cosmo_utils.helpers import ddhhmmss, make_timelist, yyyymmddhhmmss_strtotime, yyyymmddhhmmss
import timeit

ATTENTION: You are using a very early, definitely not bug-free netCDF version of getfobj. Please report bugs to s.rasp@lmu.de or fix them ;)


In [2]:
exp_ids = ['DA_REF', 'DA_PSPv2', 'DA_REF_TL500', 'DA_PSPv2_TL500', 'noDA_PSPv2']
date_start = '20160526000000'
date_stop = '20160608120000'
hint = 12
fclt = 24   # Forecast lead time

In [3]:
date_list = make_timelist(
    yyyymmddhhmmss_strtotime(date_start),
    yyyymmddhhmmss_strtotime(date_stop),
    dt.timedelta(hours=hint),
)

In [4]:
date_list[:5]

[datetime.datetime(2016, 5, 26, 0, 0),
 datetime.datetime(2016, 5, 26, 12, 0),
 datetime.datetime(2016, 5, 27, 0, 0),
 datetime.datetime(2016, 5, 27, 12, 0),
 datetime.datetime(2016, 5, 28, 0, 0)]

## Preload deterministic forecasts

In [5]:
def preload_det_prec_for_one_day(exp_id, date_str, fclt=24):
    data_dir = datadir + exp_id + '/'
    for h in range(1, fclt +1):
        hstr = ddhhmmss(dt.timedelta(hours=h))
        try:
            p = load_det(data_dir, date_str, hstr, return_array=True)
        except Exception:
            print 'File missing:', data_dir, date_str, hstr
            p = np.nan
        if h == 1:  # Allocate output array
            prec = np.ones((fclt, p.shape[0], p.shape[1])) * np.nan
        prec[h - 1] = p
    # Save
    save_fn = savedir_base + exp_id + '/prec_fields/' 
    if not os.path.exists(save_fn):
        os.makedirs(save_fn)
    save_fn += 'det_' +  date_str + '.npy'
    np.save(save_fn, prec)

In [41]:
preload_det_prec_for_one_day('DA_REF', date_start)

initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00010000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00000000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00020000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00010000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00030000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00020000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00040000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00030000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526

## Preload ensemble forecasts

In [6]:
def preload_ens_prec_for_one_day(exp_id, date_str, fclt=24):
    data_dir = datadir + exp_id + '_ens/'
    for h in range(1, fclt +1):
        hstr = ddhhmmss(dt.timedelta(hours=h))
        try:
            p = np.array(load_ens(data_dir, date_str, hstr, return_array=True))
        except Exception:
            print 'File missing:', data_dir, date_str, hstr
            p = np.nan
        if h == 1:  # Allocate output array
            prec = np.ones((fclt, p.shape[0], p.shape[1], p.shape[2])) * np.nan
        prec[h - 1] = p
    # Save
    save_fn = savedir_base + exp_id + '/prec_fields/' 
    if not os.path.exists(save_fn):
        os.makedirs(save_fn)
    save_fn += 'ens_' +  date_str + '.npy'
    np.save(save_fn, prec)

In [47]:
preload_ens_prec_for_one_day('DA_REF', date_start)

read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
reading took 0:00:03.646483
read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
reading took 0:00:04.738707
read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
reading took 0:00:06.395695
read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
reading took 0:00:04.334186
read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
reading took 0:00:04.423728
read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
reading took 0:00:04.442428
read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
reading took 0:00:04.723308
read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
reading took 0:00:04.542957
read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
re

## Loop over experiments and dates

In [48]:
for exp_id in exp_ids:
    print exp_id
    for date in date_list:
        print date
        date_str = yyyymmddhhmmss(date)
        preload_det_prec_for_one_day(exp_id, date_str)
        preload_ens_prec_for_one_day(exp_id, date_str)

DA_REF
2016-05-26 00:00:00
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00010000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00000000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00020000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00010000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00030000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00020000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00040000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00030000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/da

reading took 0:00:03.567355
2016-05-26 12:00:00
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526120000/det/lfff00010000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526120000/det/lfff00000000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526120000/det/lfff00020000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526120000/det/lfff00010000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526120000/det/lfff00030000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526120000/det/lfff00020000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526120000/det/lfff00040000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526120000/det/lfff00030000_15
initial path:  /project/meteo/w2w/A6/S.R

reading took 0:00:04.030199
2016-05-27 00:00:00
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527000000/det/lfff00010000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527000000/det/lfff00000000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527000000/det/lfff00020000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527000000/det/lfff00010000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527000000/det/lfff00030000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527000000/det/lfff00020000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527000000/det/lfff00040000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527000000/det/lfff00030000_15
initial path:  /project/meteo/w2w/A6/S.R

reading took 0:00:03.875127
2016-05-27 12:00:00
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527120000/det/lfff00010000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527120000/det/lfff00000000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527120000/det/lfff00020000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527120000/det/lfff00010000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527120000/det/lfff00030000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527120000/det/lfff00020000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527120000/det/lfff00040000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527120000/det/lfff00030000_15
initial path:  /project/meteo/w2w/A6/S.R

reading took 0:00:04.095630
2016-05-28 00:00:00
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160528000000/det/lfff00010000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160528000000/det/lfff00000000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160528000000/det/lfff00020000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160528000000/det/lfff00010000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160528000000/det/lfff00030000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160528000000/det/lfff00020000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160528000000/det/lfff00040000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160528000000/det/lfff00030000_15
initial path:  /project/meteo/w2w/A6/S.R

...

initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608000000/det/lfff00030000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608000000/det/lfff00020000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608000000/det/lfff00040000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608000000/det/lfff00030000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608000000/det/lfff00050000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608000000/det/lfff00040000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608000000/det/lfff00060000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608000000/det/lfff00050000_15
initial path:  /project/

initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608120000/det/lfff00020000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608120000/det/lfff00010000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608120000/det/lfff00030000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608120000/det/lfff00020000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608120000/det/lfff00040000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608120000/det/lfff00030000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608120000/det/lfff00050000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608120000/det/lfff00040000_15
initial path:  /project/

reading took 0:00:04.047074
read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
reading took 0:00:04.162770
noDA_PSPv2
2016-05-26 00:00:00
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2/20160526000000/det/lfff00010000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2/20160526000000/det/lfff00000000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2/20160526000000/det/lfff00020000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2/20160526000000/det/lfff00010000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2/20160526000000/det/lfff00030000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2/20160526000000/det/lfff00020000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2/20160526000000/det/lfff00040000_15


reading took 0:00:04.344571
read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
reading took 0:00:04.216893
read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
reading took 0:00:04.593591
read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
reading took 0:00:04.153214
2016-05-26 12:00:00
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2/20160526120000/det/lfff00010000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2/20160526120000/det/lfff00000000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2/20160526120000/det/lfff00020000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2/20160526120000/det/lfff00010000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2/20160526120000/det/lfff00030000_15
1h earlier:  /proje

Exception: Requested member file does not exist:/project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2_ens//20160526120000/ens016/lfff00120000_15

Some data is missing for the noDA runs. I built in an exception for these cases in the loading functions where missing files are filled with NaNs.

In [ ]:
for date in date_list:
    print date
    date_str = yyyymmddhhmmss(date)
    preload_det_prec_for_one_day('noDA_PSPv2', date_str)
    preload_ens_prec_for_one_day('noDA_PSPv2', date_str)

2016-05-26 00:00:00
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160526000000/det/lfff00010000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160526000000/det/lfff00000000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160526000000/det/lfff00020000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160526000000/det/lfff00010000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160526000000/det/lfff00030000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160526000000/det/lfff00020000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160526000000/det/lfff00040000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160526000000/det/lfff00030000_15
initial path:  /project/mete

reading took 0:00:03.887872
read to tmp-file /tmp/user/13098/tmpP1M9ndfieldobj_list_array_met-ws-720e07_20935.npy
reading took 0:00:04.050329
read to tmp-file /tmp/user/13098/tmpP1M9ndfieldobj_list_array_met-ws-720e07_20935.npy
reading took 0:00:03.923320
2016-05-26 12:00:00
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160526120000/det/lfff00010000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160526120000/det/lfff00000000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160526120000/det/lfff00020000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160526120000/det/lfff00010000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160526120000/det/lfff00030000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160526120000/det/lfff00020000_15
initial pa

2016-05-27 00:00:00
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160527000000/det/lfff00010000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160527000000/det/lfff00000000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160527000000/det/lfff00020000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160527000000/det/lfff00010000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160527000000/det/lfff00030000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160527000000/det/lfff00020000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160527000000/det/lfff00040000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160527000000/det/lfff00030000_15
initial path:  /project/mete

reading took 0:00:04.075031
read to tmp-file /tmp/user/13098/tmpP1M9ndfieldobj_list_array_met-ws-720e07_20935.npy
reading took 0:00:04.079382
read to tmp-file /tmp/user/13098/tmpP1M9ndfieldobj_list_array_met-ws-720e07_20935.npy
reading took 0:00:04.063613
2016-05-27 12:00:00
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160527120000/det/lfff00010000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160527120000/det/lfff00000000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160527120000/det/lfff00020000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160527120000/det/lfff00010000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160527120000/det/lfff00030000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160527120000/det/lfff00020000_15
initial pa

reading took 0:00:04.123680
read to tmp-file /tmp/user/13098/tmpP1M9ndfieldobj_list_array_met-ws-720e07_20935.npy
reading took 0:00:04.159870
read to tmp-file /tmp/user/13098/tmpP1M9ndfieldobj_list_array_met-ws-720e07_20935.npy
reading took 0:00:04.160862
read to tmp-file /tmp/user/13098/tmpP1M9ndfieldobj_list_array_met-ws-720e07_20935.npy
reading took 0:00:04.067427
read to tmp-file /tmp/user/13098/tmpP1M9ndfieldobj_list_array_met-ws-720e07_20935.npy
reading took 0:00:04.015905
2016-05-28 00:00:00
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160528000000/det/lfff00010000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160528000000/det/lfff00000000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160528000000/det/lfff00020000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160528000000/det/lfff00010000_15
initial path:  /proj

...

initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160605000000/det/lfff00180000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160605000000/det/lfff00170000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160605000000/det/lfff00190000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160605000000/det/lfff00180000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160605000000/det/lfff00200000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160605000000/det/lfff00190000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160605000000/det/lfff00210000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160605000000/det/lfff00200000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kend

initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160605120000/det/lfff00170000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160605120000/det/lfff00160000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160605120000/det/lfff00180000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160605120000/det/lfff00170000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160605120000/det/lfff00190000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160605120000/det/lfff00180000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160605120000/det/lfff00200000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160605120000/det/lfff00190000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kend

initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160606000000/det/lfff00160000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160606000000/det/lfff00150000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160606000000/det/lfff00170000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160606000000/det/lfff00160000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160606000000/det/lfff00180000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160606000000/det/lfff00170000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160606000000/det/lfff00190000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160606000000/det/lfff00180000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kend

initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160606120000/det/lfff00150000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160606120000/det/lfff00140000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160606120000/det/lfff00160000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160606120000/det/lfff00150000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160606120000/det/lfff00170000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160606120000/det/lfff00160000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160606120000/det/lfff00180000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160606120000/det/lfff00170000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kend

initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160607000000/det/lfff00140000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160607000000/det/lfff00130000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160607000000/det/lfff00150000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160607000000/det/lfff00140000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160607000000/det/lfff00160000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160607000000/det/lfff00150000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160607000000/det/lfff00170000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2//20160607000000/det/lfff00160000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kend

## Preload radar observations

In [15]:
def preload_radar_prec_for_one_day(date_str, fclt=24):
    for h in range(1, fclt +1):
        hstr = ddhhmmss(dt.timedelta(hours=h))
        p = load_radar(date_str, hstr, return_array=True);
        if h == 1:  # Allocate output array
            prec = np.empty((fclt, p.shape[0], p.shape[1]))
        prec[h - 1] = p
    # Save
    save_fn = savedir_base + 'radar/prec_fields/' 
    if not os.path.exists(save_fn):
        os.makedirs(save_fn)
    save_fn += 'radar_' +  date_str + '.npy'
    np.save(save_fn, prec)

In [16]:
preload_radar_prec_for_one_day(date_start)

In [17]:
for date in date_list:
    print date
    date_str = yyyymmddhhmmss(date)
    preload_radar_prec_for_one_day(date_str)

2016-05-26 00:00:00
2016-05-26 12:00:00
2016-05-27 00:00:00
2016-05-27 12:00:00
2016-05-28 00:00:00
2016-05-28 12:00:00
2016-05-29 00:00:00
2016-05-29 12:00:00
2016-05-30 00:00:00
2016-05-30 12:00:00
2016-05-31 00:00:00
2016-05-31 12:00:00
2016-06-01 00:00:00
2016-06-01 12:00:00
2016-06-02 00:00:00
2016-06-02 12:00:00
2016-06-03 00:00:00
2016-06-03 12:00:00
2016-06-04 00:00:00
2016-06-04 12:00:00
2016-06-05 00:00:00
2016-06-05 12:00:00
2016-06-06 00:00:00
2016-06-06 12:00:00
2016-06-07 00:00:00
2016-06-07 12:00:00
2016-06-08 00:00:00
2016-06-08 12:00:00


## Preload CAPE and CIN

In [ ]:
def preload_det_cape_cin_for_one_day(exp_id, date_str, fclt=24):
    data_dir = datadir + exp_id + '/'
    for h in range(1, fclt +1):
        hstr = ddhhmmss(dt.timedelta(hours=h))
        try:
            ca, ci = load_det_cape_cin(data_dir, date_str, hstr, return_array=True)
        except Exception:
            print 'File missing:', data_dir, date_str, hstr
            ca, ci = (np.nan, np.nan)
        if h == 1:  # Allocate output array
            cape = np.ones((fclt, ca.shape[0], ca.shape[1])) * np.nan
            cin = np.ones((fclt, ca.shape[0], ca.shape[1])) * np.nan
        cape[h - 1] = ca
        cin[h - 1] = ci
    # Save
    save_fn_cape = savedir_base + exp_id + '/cape_fields/' 
    if not os.path.exists(save_fn_cape):
        os.makedirs(save_fn_cape)
    save_fn += 'det_' +  date_str + '.npy'
    np.save(save_fn, cape)
    save_fn_cin = savedir_base + exp_id + '/cin_fields/' 
    if not os.path.exists(save_fn_cin):
        os.makedirs(save_fn_cin)
    save_fn += 'det_' +  date_str + '.npy'
    np.save(save_fn, cin)

In [ ]:
for exp_id in exp_ids:
    print exp_id
    for date in date_list:
        print date
        date_str = yyyymmddhhmmss(date)
        preload_det_cape_cin_for_one_day(exp_id, date_str)